In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchvision
import numpy as np
from tqdm import tqdm
from arguments import get_args
from augmentations import get_aug
from models import get_model
from tools import AverageMeter, knn_monitor, Logger, file_exist_check
from datasets import get_dataset
from optimizers import get_optimizer, LR_Scheduler
from linear_eval import main as linear_eval
#from datetime import datetime
import numpy as np
import time
import collections
from torch import optim
import torch
from sklearn import metrics, preprocessing
import datetime
from sklearn.decomposition import PCA

import sys
sys.path.append('./global_module/')
import network
import train
from generate_pic import aa_and_each_accuracy, sampling,load_dataset, generate_png1, generate_iter
from Utils import record, extract_samll_cubic
import byol_pytorch
import vit

In [2]:
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
def generate_iter_ViT(TRAIN_SIZE, train_indices, TEST_SIZE, test_indices, TOTAL_SIZE, total_indices, VAL_SIZE,
                  whole_data, PATCH_LENGTH, padded_data, INPUT_DIMENSION, batch_size, gt,args=None):

    gt_all = gt[total_indices] - 1

    all_data = extract_samll_cubic.select_small_cubic(TOTAL_SIZE, total_indices, whole_data,
                                                      PATCH_LENGTH, padded_data, INPUT_DIMENSION)

    all_data.reshape(all_data.shape[0], all_data.shape[1], all_data.shape[2], INPUT_DIMENSION)
    all_tensor_data = torch.from_numpy(all_data).type(torch.FloatTensor).permute(0,3,1,2)
    all_tensor_data_label = torch.from_numpy(gt_all).type(torch.FloatTensor)
    #print(all_tensor_data.shape)
    #print(all_tensor_data_label.shape)
    torch_dataset_all = Data.TensorDataset(all_tensor_data, all_tensor_data_label)

    trainall_iter = Data.DataLoader(
            dataset=torch_dataset_all,  # torch TensorDataset format
            batch_size=batch_size,  # mini batch size
            shuffle=True,  # 要不要打乱数据 (打乱比较好)
            num_workers=0,  # 多线程来读数据
        )
    all_iter = Data.DataLoader(
        dataset=torch_dataset_all,  # torch TensorDataset format
        batch_size=batch_size,  # mini batch size
        shuffle=False,  # 要不要打乱数据 (打乱比较好)
        num_workers=0,  # 多线程来读数据
    )
    return all_iter,trainall_iter #, y_test

In [3]:
import random
#数据增强

def applyPCA(X, numComponents=15):
        newX = np.reshape(X, (-1, X.shape[2]))
        pca = PCA(n_components=numComponents, whiten=True,svd_solver='full')#不加full会报数组错误
        newX = pca.fit_transform(newX)
        newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
        return newX

def stochastic_augment(x):
    num = random.randint(0,7)
    if num >= 0 and num < 4:
        #print(x.shape)
        aug = np.fliplr(x)
        aug=aug.transpose(1,2,0)
        np.random.seed(int(time.time()))#假如不设置随机数！！！！！！！！！！
        #aug = cv2.GaussianBlur(np.float32(aug), (7, 7), wz)
        noise=np.random.randint(0,2,(aug.shape[0],aug.shape[1]))
        noise[aug.shape[0]//2,aug.shape[1]//2]=1#确保中心没有噪声
        noise=noise[:,:,np.newaxis]
        noise=np.concatenate([noise]*aug.shape[2],2)
        aug=aug*noise
        aug=aug.transpose(2,0,1)
        Aug = torch.from_numpy(aug.copy())

    elif num > 3 and num < 8:
        #print(x.shape)
        aug = np.flipud(x)
        aug=aug.transpose(1,2,0)
        np.random.seed(int(time.time()))#假如不设置随机数！！！！！！！！！！！！！
        #aug = cv2.GaussianBlur(np.float32(aug), (7, 7), wz)
        noise=np.random.randint(0,2,(aug.shape[0],aug.shape[1]))
        noise[aug.shape[0]//2,aug.shape[1]//2]=1#确保中心没有噪声
        noise=noise[:,:,np.newaxis]
        noise=np.concatenate([noise]*aug.shape[2],2)
        aug=aug*noise
        aug=aug.transpose(2,0,1)
        Aug = torch.from_numpy(aug.copy())
    else:
        Aug = x
    return Aug

def stochastic_augment_v2(x):
    num = random.randint(0,7)
    if num >= 0 and num < 4:
        #print(x.shape)
        aug = np.fliplr(x)
        aug=aug.transpose(1,2,0)
        np.random.seed(int(time.time()))#加入不设置随机数！！！！！！！！！！！！！！！
        #aug = cv2.GaussianBlur(np.float32(aug), (7, 7), wz)
        noise=np.ones((aug.shape[0],aug.shape[1]))
        ran1=np.random.randint(0,aug.shape[0]-1)
        ran2=np.random.randint(ran1+1,aug.shape[0])
        ran3=np.random.randint(0,aug.shape[0]-1)
        ran4=np.random.randint(ran3+1,aug.shape[0])
        noise[ran1:ran2,ran3:ran4]=0
        noise[aug.shape[0]//2,aug.shape[1]//2]=1#确保中心没有噪声
        noise=noise[:,:,np.newaxis]
        noise=np.concatenate([noise]*aug.shape[2],2)
        aug=aug*noise
        aug=aug.transpose(2,0,1)
        Aug = torch.from_numpy(aug.copy())
    elif num >3 and num < 8:
        #print(x.shape)
        aug = np.flipud(x)
        aug=aug.transpose(1,2,0)
        np.random.seed(int(time.time()))#假如不设置随机数！！！！！！！！！！！！！！！！
        #aug = cv2.GaussianBlur(np.float32(aug), (7, 7), wz)
        noise=np.ones((aug.shape[0],aug.shape[1]))
        ran1=np.random.randint(0,aug.shape[0]-1)
        ran2=np.random.randint(ran1+1,aug.shape[0])
        ran3=np.random.randint(0,aug.shape[0]-1)
        ran4=np.random.randint(ran3+1,aug.shape[0])
        noise[ran1:ran2,ran3:ran4]=0
        noise[aug.shape[0]//2,aug.shape[1]//2]=1#确保中心没有噪声
        noise=noise[:,:,np.newaxis]
        noise=np.concatenate([noise]*aug.shape[2],2)
        aug=aug*noise
        aug=aug.transpose(2,0,1)
        Aug = torch.from_numpy(aug.copy())
    return Aug

def argument3(data):#，3D空间随机掩码,ratio为掩码率
        np.random.seed(int(time.time()))
        a = torch.rand_like(data)
        zero = torch.zeros_like(data)
        one = torch.ones_like(data)
        b = torch.where(a > 0.1, one, zero) 
        return data*b

def argument4(data):#，3D空间随机掩码,ratio为掩码率，中心像素不掩码
        np.random.seed(int(time.time()))
        a = torch.rand_like(data)
        zero = torch.zeros_like(data)
        one = torch.ones_like(data)
        b = torch.where(a > 0.10, one, zero)
        b[:,b.shape[1]//2,b.shape[2]//2] = 1 
        return data*b

def argument5(data):#，3D空间随机掩码,ratio为掩码率，光广播中心像素不掩码,range为广播范围!!!!
        range=3
        ratio=0.1
        #中心像素先广播
        centre = data[:,data.shape[1]//2,data.shape[1]//2]#中心像素[band]
        centre2=centre.view(data.shape[0],1,1)#改变维度[band,1,1]
        centre3=centre2.repeat(1,range,range)#centre变为[band,rang,rang]
        data[:,data.shape[1]//2-range//2:data.shape[1]//2+range//2+1,data.shape[1]//2-range//2:data.shape[1]//2+range//2+1]=centre3#中心像素广播成功！！！！！！！
        
        #3D掩码，广播像素不掩码
        a = torch.rand_like(data)
        zero = torch.zeros_like(data)
        one = torch.ones_like(data)
        b = torch.where(a > ratio, one, zero)
        b[:,b.shape[1]//2-range//2:b.shape[1]//2+range//2+1,b.shape[1]//2-range//2:b.shape[1]//2+range//2+1] = 1 
        return data*b

def radiation_noise(data, alpha_range=(0.6, 1.4), beta=1/5):
        alpha = np.random.uniform(*alpha_range)
        noise = np.random.normal(loc=0., scale=1.0, size=data.shape)
        return alpha * data + beta * noise

In [4]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--device",default=torch.device('cuda:0'),help="The model device")
parser.add_argument('--Dataset', default='sv',help='up,sv ,in or ksc')
parser.add_argument('--lr', default= 1e-3 , help='the learning rate')
parser.add_argument('--epochs', default=20)
parser.add_argument('--batch_size', default=256)
parser.add_argument('--name',default='byolup',help='保存的模型名')
parser.add_argument('--val_split',default=0.1,help='测试集占多少')#不用这个了
args = parser.parse_args(args=[])
print(args)

Namespace(Dataset='sv', batch_size=256, device=device(type='cuda', index=0), epochs=20, lr=0.001, name='byolup', val_split=0.1)


In [5]:
device = args.device
seeds = [1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341]
day = datetime.datetime.now()
day_str = day.strftime('%m_%d_%H_%M')

print('-----Importing Dataset-----')
global Dataset  # UP,IN,KSC
dataset = args.Dataset
#dataset='up'
Dataset = dataset.upper()
data_hsi, gt_hsi, TOTAL_SIZE, TRAIN_SIZE, VALIDATION_SPLIT = load_dataset(Dataset)
print("###########data_hsi.shape###############")
print(data_hsi.shape)
data_hsi =applyPCA(data_hsi,15)
print("###########PCA_data_hsi.shape###############")
print(data_hsi.shape)
image_x, image_y, BAND = data_hsi.shape
data = data_hsi.reshape(np.prod(data_hsi.shape[:2]), np.prod(data_hsi.shape[2:]))
print("$$$$$$$$$$$data.shape$$$$$$$$$$$$$$")
print(data.shape)
gt = gt_hsi.reshape(np.prod(gt_hsi.shape[:2]),)
CLASSES_NUM = max(gt)
print('The class numbers of the HSI data is:', CLASSES_NUM)

print('-----Importing Setting Parameters-----')
ITER = 1
PATCH_LENGTH =13
# number of training samples per class
#lr, num_epochs, batch_size = 0.0010, 200, 32
lr, num_epochs, batch_size = args.lr, args.epochs, args.batch_size

# loss = torch.nn.CrossEntropyLoss()#将这里注释，看看有什么问题

img_rows = 2*PATCH_LENGTH+1
img_cols = 2*PATCH_LENGTH+1
img_channels = data_hsi.shape[2]
INPUT_DIMENSION = data_hsi.shape[2]
ALL_SIZE = data_hsi.shape[0] * data_hsi.shape[1]
VAL_SIZE = int(TRAIN_SIZE)
TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE

KAPPA = []
OA = []
AA = []
TRAINING_TIME = []
TESTING_TIME = []
ELEMENT_ACC = np.zeros((ITER, CLASSES_NUM))

data = preprocessing.scale(data)
data_ = data.reshape(data_hsi.shape[0], data_hsi.shape[1], data_hsi.shape[2])
whole_data = data_
padded_data = np.lib.pad(whole_data, ((PATCH_LENGTH, PATCH_LENGTH), (PATCH_LENGTH, PATCH_LENGTH), (0, 0)),
                         'constant', constant_values=0)

for index_iter in range(ITER):
    print('iter:', index_iter)
    #net = network.FDSSC_network(BAND, CLASSES_NUM)
        # define model
    #model=vit.ViT(image_size = PATCH_LENGTH*2+1,patch_size = 5,num_classes = CLASSES_NUM,dim = 1024,depth = 4,heads = 16,mlp_dim = 2048,
    model=vit.ViT(image_size = PATCH_LENGTH*2+1,patch_size = 3,dim = 256,depth = 2,heads = 16,mlp_dim =512,
                  pool = 'cls', channels = BAND, dim_head = 32,dropout = 0.,emb_dropout = 0.,device=device) 
    #model=torch.load('./results/byol_0122212043.pt')
    #model.load_state_dict(torch.load('./results/byol_0122203240.pt', map_location=device))
    model=model.to(device)
    '''
    learner=byol_pytorch.BYOL(model,image_size=PATCH_LENGTH*2+1,channels=BAND,hidden_layer = -2,projection_size = 512,
        projection_hidden_size = 1024,
        augment_fn = stochastic_augment,
        augment_fn2 = stochastic_augment_v2,
        moving_average_decay = 0.99,
        use_momentum = True)
    '''
    learner=byol_pytorch.BYOL(model,image_size=PATCH_LENGTH*2+1,channels=BAND,hidden_layer = -1,projection_size =64,
        projection_hidden_size =1024,
        # augment_fn = stochastic_augment,
        # augment_fn2 = stochastic_augment_v2,
        augment_fn = argument4,
        augment_fn2 = argument4,
        moving_average_decay = 0.99,
        use_momentum = True)
    optimizer = optim.Adam(learner.parameters(), lr=lr)  # , weight_decay=0.0001)
    # optimizer = optim.SGD(learner.parameters(), lr=lr)  # , weight_decay=0.0001)
    time_1 = int(time.time())
    np.random.seed(seeds[index_iter])
    train_indices, test_indices = sampling(args.val_split, gt)
    _, total_indices = sampling(1, gt)

    TRAIN_SIZE = len(train_indices)
    print('Train size: ', TRAIN_SIZE)
    TEST_SIZE = TOTAL_SIZE - TRAIN_SIZE
    print('Test size: ', TEST_SIZE)
    VAL_SIZE = 100#实际上没用，我没用这个
    print('Validation size: ', VAL_SIZE)

    print('-----Selecting Small Pieces from the Original Cube Data-----')

    all_iter,trainall_iter = generate_iter_ViT(TRAIN_SIZE, train_indices, TEST_SIZE, test_indices, TOTAL_SIZE, total_indices, VAL_SIZE,
                  whole_data, PATCH_LENGTH, padded_data, INPUT_DIMENSION, batch_size, gt)
    # Start training
    global_progress = tqdm(range(0, num_epochs), desc=f'Training')
    loss_record=[]
    for epoch in global_progress:
        model.train()
        
        local_progress=tqdm(trainall_iter, desc=f'Epoch {epoch}/{num_epochs}')#用所有的进行训练
        loss_epoch=0
        for idx, (images, labels) in enumerate(local_progress):
            #print(images.shape)
            loss = learner(images)
            optimizer.zero_grad()
            loss_epoch=loss_epoch+loss.item()
            loss.backward()
            optimizer.step()
            learner.update_moving_average() # update moving average of target encoder
        loss_record.append(loss_epoch)
        if loss_epoch<=min(loss_record):
            print('最小loss，保存模型',loss_epoch)
            # model_path = os.path.join('./results/', f"{args.name}_{datetime.datetime.now().strftime('%m%d%H%M%S')}_epoch{epoch}.pt") # datetime.now().strftime('%Y%m%d_%H%M%S')
            model_path = os.path.join('./results/byolsv_a44_mask0.1.pt')#！！！！！！！！！保存模型！！！！！！！！！！！！！！！！
            torch.save(model.state_dict(),model_path)

        #if args.train.knn_monitor and epoch % args.train.knn_interval == 0: 
        #if True and epoch % 1 ==0:
            #accuracy = knn_monitor(model.module.backbone, memory_loader, test_loader, device, k=min(200, len(memory_loader.dataset)), hide_progress=args.hide_progress) 
        
        #epoch_dict = {"epoch":epoch, "accuracy":accuracy}
        #global_progress.set_postfix(epoch_dict)
        #logger.update_scalers(epoch_dict)
    print('训练结束',loss_epoch)
   


-----Importing Dataset-----
###########data_hsi.shape###############
(512, 217, 204)
###########PCA_data_hsi.shape###############
(512, 217, 15)
$$$$$$$$$$$data.shape$$$$$$$$$$$$$$
(111104, 15)
The class numbers of the HSI data is: 16
-----Importing Setting Parameters-----
iter: 0
Train size:  48714
Test size:  5415
Validation size:  100
-----Selecting Small Pieces from the Original Cube Data-----


Training:   5%|▌         | 1/20 [00:57<18:13, 57.55s/it]

最小loss，保存模型 19.595827873796225


Training:  10%|█         | 2/20 [01:54<17:15, 57.52s/it]

最小loss，保存模型 1.0258496506139636


Training:  15%|█▌        | 3/20 [02:52<16:18, 57.58s/it]

最小loss，保存模型 0.8157685147598386


Training: 100%|██████████| 20/20 [19:18<00:00, 57.93s/it]

训练结束 2.4429661091417074


In [6]:
import matplotlib
matplotlib.matplotlib_fname()
print(matplotlib.get_backend())

print(matplotlib.matplotlib_fname())
import matplotlib.rcsetup as rcsetup
print(rcsetup.all_backends)

agg
/root/anaconda3/envs/lzz/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc
['GTK3Agg', 'GTK3Cairo', 'MacOSX', 'nbAgg', 'Qt4Agg', 'Qt4Cairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']
